# Locality Sensitive Hashing

In [ ]:
from matplotlib.pyplot import figure as fg
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import random
import numpy as np
import binascii

In [ ]:
articles = pd.read_csv('preproc_art_small.csv')

In [ ]:
encoded_c_p = dict()
encoded_p_c = dict()

In [ ]:
# Encodes a type in a sequencial int
def encode_property(property_name, plaintext):
  global encoded_c_p, encoded_p_c
  code = 0
  plaintext_str = str(plaintext)
  try:
    tmp = encoded_p_c[property_name] #Check if property is in dict
    try:
      code = tmp[plaintext_str] #Check if category is in dict
    except:
      encoded_p_c[property_name][plaintext_str] = binascii.crc32(bytes(plaintext,'utf-8'))
      encoded_c_p[property_name][encoded_p_c[property_name][plaintext_str]] = plaintext_str
  except:
    encoded_c_p[property_name] = dict()
    encoded_p_c[property_name] = dict()
    encoded_p_c[property_name][plaintext_str] = binascii.crc32(bytes(plaintext,'utf-8'))
    encoded_c_p[property_name][encoded_p_c[property_name][plaintext_str]] = plaintext_str
  code = encoded_p_c[property_name][plaintext_str]
  return code

def decode_property(property_name, code):
  global encoded_c_p
  tmp = encoded_c_p[property_name] #Check if property is in dict
  plaintext = tmp[code] #Check if category is in dict
  return plaintext

# Shingles

In [ ]:
shingles=list()

In [ ]:
list_docs = list(articles.article.apply(str))

In [ ]:
# Function creates n-gram
def crete_grams(doc, n = 3):
    grams = []
    for tokens_len in range(len(doc.split())):
        if tokens_len + n > len(doc.split()):
            break
        grams.append(' '.join(doc.split()[tokens_len : tokens_len+n]))
    return grams

In [ ]:
# Here we create the shingles of all the text
for doc in tqdm(list_docs):
    shingles.extend(crete_grams(doc, n=5))

In [ ]:
# Here the shingles are encoded in  32 bit integer
for gram in tqdm(shingles):
    encode_property('sh',gram)

In [ ]:
# This function given a text will return the codes corresponding to each of its shingles
def convert_text_in_shingles(text, n_grams = 3):
    shingles = crete_grams(text, n_grams)
    hashed_text = []
    for shingle in shingles:
        hashed_text.append(encode_property('sh', shingle)) # Encode, if the word is already present returns the corresponding words
    return hashed_text


In [ ]:
articles['sh'] = articles.article.progress_apply(lambda x: convert_text_in_shingles(str(x), 5))

# SignMatrix

In [ ]:
#Maximum used shingle hash
max_sh_ID = 2**32-1
print(max_sh_ID)

In [ ]:
#used to calculate a, b
def rand_k_coeff(k):
    ret_list = []
    for x in range(k):
        found = True
        while(found):
            ran_int = random.randint(0, max_sh_ID)
            if ran_int not in ret_list:
                ret_list.append(ran_int)
                found = False
    return ret_list

In [ ]:
# http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
next_prime = 4294967311

In [ ]:
#ret (ax+b)%next_prime, next_prime:max shingle index, x: shingle id a,b:rand num
def get_hash_funcs(shingles, n_esec, a, b):
    global next_prime
    min_hash = next_prime +1
    for shingle in shingles:
        hash_code = ((a[n_esec] * shingle + b[n_esec]) % next_prime)
        if hash_code < min_hash:
            min_hash = hash_code
    return int(min_hash)

calculate signature matrix

In [ ]:
def create_sign_matr(num_hash_fun, docs_df):
    a = rand_k_coeff(num_hash_fun)
    b = rand_k_coeff(num_hash_fun)
    signature_matr = np.zeros((num_hash_fun, len(list_docs)), int)
    for y in tqdm(range(num_hash_fun)):
        signature_matr[y, :] = (docs_df['sh'].apply(lambda x: get_hash_funcs(x, y, a, b)))
    return signature_matr

# LSH

In [ ]:
# insert an element in the correct bucket
def check_bucket(bucket_hash, hash, id):
    try: # If there is already this hash in the bucket we will add the id
        similar_list = bucket_hash[hash]
        similar_list.append(id)
        bucket_hash[hash] = similar_list
    except:# Otherwise we create a cell in the bucket for this hash
        bucket_hash[hash] = [id]
    return bucket_hash


In [ ]:
from itertools import combinations
# Construct the buckets
def get_buckets_LSH(sign_matr, band, rows):
    buckets_id = []
    num_hash_fun = sign_matr.shape[0]
    num_docs = sign_matr.shape[1]
    tmp_sign = sign_matr[:(band*rows),:].copy()
    if band*rows > num_hash_fun:
        return 0
    for band in range(band):
        actual_band = tmp_sign[band*rows:(band+1)*rows,:] #get the band hashes
        bucket_hash = dict()
        for doc_hash_band in range(num_docs): #Iterate on the hashes
            final_hash = ''.join(list(map(str,list(actual_band[:, doc_hash_band]))))
            bucket_hash = check_bucket(bucket_hash, final_hash, doc_hash_band)
        bucket_hash = {k: v for k, v in bucket_hash.items() if len(v) >= 2}
        buckets_id.append(bucket_hash)
    return buckets_id

In [ ]:
from itertools import combinations, groupby
# Resturns from the buckets the list of candidate pairs
def get_candidate_pairs(buckets):
    pairs = []
    if buckets == 0:
        return pairs
    for bucket in buckets:
        pairs.extend(bucket.values())
    pairs.sort()
    pairs = list(k for k,_ in groupby(pairs))
    pairs_2 = list(filter(lambda x: len(x) <3, pairs))
    pairs_3 = list(filter(lambda x: len(x) >2, pairs))
    tmp_pairs = []
    for x in pairs_3:
        tmp_pairs.extend(list(map(list, list(combinations(x, 2)))))
    pairs_2.extend(tmp_pairs)
    pairs_2.sort()
    pairs = list(k for k,_ in groupby(pairs_2))
    return pairs

# Evaluate

In [ ]:
g_truth = np.load('g_truth.npy')

In [ ]:
def evaluate(g_truth, candidates, similarity): # calculates tp, tn, fp, fn
    tp = 0
    fn = 0
    truth_indexes = np.argwhere(g_truth >= similarity)
    false_indexes = len(np.argwhere(g_truth < similarity)) - int(((g_truth.shape[0]**2)-g_truth.shape[0])/2) - g_truth.shape[0] 
    for index in (truth_indexes):
        if len(candidates) == 0:
            return [0,0,0]
        ret_list = list(candidates[(candidates[0] == index[0])][1])
        ret_list.extend(list(candidates[(candidates[1] == index[0])][0]))
        if index[1] in ret_list:
            tp += 1
        else:
            fn += 1
    fp = len(candidates) - tp
    tn = false_indexes - fn
    return [tp, fp, fn, tn]

In [ ]:
precision = lambda tp, fp: tp/(tp+fp)
sensitivity = lambda tp, fn: tp/(tp + fn)
specificity = lambda tn, fp: tn/(tn+fp)
falseposrate = lambda fp, tn: fp/(fp+tn)
falsenegrate = lambda fn, tp: fn/(fn+tp)
accuracy = lambda tn, tp, fn, fp: (tp+tn)/(tp+tn+fp+fn)
f1 = lambda tp, fn, fp: (2*tp)/((2*tp)+fp+fn)

In [ ]:
def get_all_score(tp, fp, fn, tn):
    global precision, specificity, sensitivity, accuracy, falseposrate, f1, falsenegrate
    print("Precision:\t " + str(round(precision(tp, fp)*100,2)))
    print("F1:\t\t " + str(round(f1(tp, fn, fp)*100,2)))
    print("Accuracy:\t " + str(round(accuracy(tn, tp, fn, fp)*100,2)))
    print("FPR:\t\t " + str(round(falseposrate(fp, tn)*100,2)))
    print("FNR:\t\t " + str(round(falsenegrate(fn, tp)*100,2)))
    print("Specificity:\t " + str(round(specificity(tn, fp)*100,2)))
    print("Sensitivity:\t " + str(round(sensitivity(tp, fn)*100,2)))


In [ ]:
def get_best_res(res, w_prec, w_spec, w_sens): # Use a weighted mean to select the best result
    res['mean'] = res.iloc[:,3:6].apply(lambda x: ((w_prec*x[0])+(w_spec*x[1])+(w_sens*x[2]))/(w_prec+w_spec+w_sens), axis = 1)
    res = res.sort_values(by=['mean'], ascending=False).reset_index(drop=True)
    return res.head(1)

# Grid Search

In [ ]:
import math

In [ ]:
signature_matrix = create_sign_matr(1000, articles)

In [ ]:
test_sim = [0.5, 0.6, 0.7, 0.8, 0.9]
test_R = range(3,11)

In [ ]:
#perform tests on the combination of different number of rows, returns the scores prec, sens, spec
def grid_search(test_R, p1, tolerance, signature_matrix, g_truth, similarity = 0.8):
    res_matr = np.zeros((1, 6))
    count = 0
    for testing_R in tqdm(test_R):
        testing_L = round(math.log(1-p1, (1-((similarity-tolerance)**testing_R)))) #calculates R
        if(testing_L*testing_R > signature_matrix.shape[0]): continue # Verify the selected number of bands is coherent
        tmp_signature = signature_matrix[:(testing_L*testing_R),:].copy()
        # Perform LSH
        buckets = get_buckets_LSH(tmp_signature, rows = testing_R, band=testing_L)
        pairs = get_candidate_pairs(buckets)
        if(len(pairs) < 1): continue
        pairs = pd.DataFrame(pairs)
        tp, fp, fn, tn = evaluate(g_truth, pairs, similarity)
        p = round(precision(tp, fp)*100,2)
        sen = round(sensitivity(tp, fn)*100,2)
        spe = round(specificity(tn, fp)*100,3)
        res_matr = np.vstack([res_matr, [(testing_L*testing_R), testing_R, testing_L, p, spe, sen]])
    count +=1
    res_matr = res_matr[1:,:]
    return pd.DataFrame(res_matr, columns =['Num_Hash', 'Rows', 'Bands', 'Prec', 'Spec', 'Sens'])

In [ ]:
tol = 0.15
analysis_dict = {}
for x in test_sim:
    res = grid_search(test_R, 0.95, tol, signature_matrix, g_truth, similarity=x)
    analysis_dict[x*100] = get_best_res(res, 3,1,2).iloc[0].tolist()
print(analysis_dict)

# Plot

In [ ]:
fg(figsize=(12, 8), dpi=80)
x = list(map(int, analysis_dict.keys()))
y_sen = [d[5] for d in analysis_dict.values()]
y_spec = [d[4] for d in analysis_dict.values()]
y_prec = [d[3] for d in analysis_dict.values()]
plt_hash = [d[0] for d in analysis_dict.values()]
plt_rows = [d[1] for d in analysis_dict.values()]
plt_bands = [d[2] for d in analysis_dict.values()]
width = 0.80

# plot data in grouped manner of bar type
for ind in range(len(x)):
    plt.bar(x[ind]-0.9, y_prec[ind], width, color = 'blue')
    plt.bar(x[ind]+0.9, y_spec[ind], width, color = 'green')
    plt.bar(x[ind], y_sen[ind], width, color = 'orange')
plt.ylabel('%')
for i in range(len(x)):
    plt.text(x[i]-1.2, y_prec[i]+1.5, str(y_prec[i]) + '%', color='blue', fontweight='bold', rotation = 90)
    plt.text(x[i]+.7, y_spec[i]+1.5, str(y_spec[i]) + '%', color='green', fontweight='bold', rotation = 90)
    plt.text(x[i]-0.2, y_sen[i]+1.5, str(y_sen[i]) + '%', color='orange', fontweight='bold', rotation = 90)

    plt.text(x[i]+1.5, 50+13, "N° Hash:\n "+str(plt_hash[i]), color='black', fontweight='bold')
    plt.text(x[i]+1.5, 50+7, "N° Bands:\n " + str(plt_bands[i]), color='black', fontweight='bold')
    plt.text(x[i]+1.5, 50+1, "N° Rows:\n " + str(plt_rows[i]), color='black', fontweight='bold')
    p1 = str( 1-(1-((x[i]/100 - tol)**plt_rows[i]))**plt_bands[i] )[:4]
    p2 = str(1-(1-((x[i]/100)**plt_rows[i]))**plt_bands[i])[:4]
    s1 = str(x[i] - (tol*100))
    plt.text(x[i]+1.5, 50-5, "(" + s1 + "%, " + str(x[i]) + "%,\n" + p1 + ", " + p2+")", color='black', fontweight='bold')
plt.legend(["Prec", "Spec", "Sens"], loc = 'lower left')
plt.xlabel('%Similarity')

In [ ]:
# Insert in final_sum for each similarity range (e.g., 0.0 - 0.1, ...) the count of pairs in that range
final_sum = []
step = 0.1
sim_range = np.arange(0.0, 1.0, step)
for sim_step in sim_range:
    next_step = sim_step + step
    if sim_step == 0.0:
        elems = np.argwhere((g_truth >= sim_step) & (g_truth < next_step))
        final_sum.append(len(elems) - 499500)
        continue
    elems = np.argwhere((g_truth >= sim_step) & (g_truth < next_step))
    final_sum.append(len(elems))

In [ ]:
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

In [ ]:
plt.style.use('ggplot')
fig = plt.figure(figsize=(10, 15), dpi=80)
gs = fig.add_gridspec(3, hspace=0.1)
axs = gs.subplots(sharex=True, sharey=True)

x_bar = [0,10,20,30,40,50,60,70,80,90]
y_bar = final_sum

#p1 (65, 80, 95, 99)
x = np.arange(0, 100, 0.01)
y_outputs = (list(map(round_down,(1-(1-(x/100)**8)**93)*100)))

ax02 = axs[0].twinx()

axs[0].bar(x_bar, y_bar, color ='blue', width = 9.5, align='edge', log=10)

for v in range(len(final_sum)):
    axs[0].text(x_bar[v]+3.5, np.log10(y_bar[v])*9, str(final_sum[v]), color='orange', fontweight='bold', rotation = 90, size = 20)

ax02.axvline(x=65, color = 'blue', linestyle = ':')
ax02.axvline(x=80, color = 'blue', linestyle = ':')
ax02.axhline(y=95, color = 'blue', linestyle = ':')
ax02.axhline(y=99, color = 'blue', linestyle = ':')



ax02.plot(x, y_outputs)
#ax02.set_ylim([0, ymax])
ax02.text(67,50, 'Tolerance', color='blue', fontweight='bold')
ax02.text(53,30, 'FalsePositive', color='blue', fontweight='bold')
ax02.text(82,101, 'FalseNegative', color='blue', fontweight='bold')
ax02.set_title('(65%,80%,95%,99%)')

#p2 (75, 80, 95, 99)
x = np.arange(0, 100, 0.01)
y_outputs = (list(map(round_down,(1-(1-(x/100)**10)**52)*100)))

ax12 = axs[1].twinx()

axs[1].bar(x_bar, y_bar, color ='blue', width = 9.5, align='edge', log=10)

for v in range(len(final_sum)):
    axs[1].text(x_bar[v]+3.5, np.log10(y_bar[v])*9, str(final_sum[v]), color='orange', fontweight='bold', rotation = 90, size = 20)

ax12.axvline(x=75, color = 'blue', linestyle = ':')
ax12.axvline(x=80, color = 'blue', linestyle = ':')
ax12.axhline(y=95, color = 'blue', linestyle = ':')
ax12.axhline(y=99, color = 'blue', linestyle = ':')

ax12.plot(x, y_outputs)
ax12.text(70,50, 'Tolerance', color='blue', fontweight='bold')
ax12.text(60,30, 'FalsePositive', color='blue', fontweight='bold')
ax12.text(82,101, 'FalseNegative', color='blue', fontweight='bold')
ax12.set_title('(75%,80%,95%,99%)')

#p3 (50, 80, 85, 99)
x = np.arange(0, 100, 0.01)
y_outputs = (list(map(round_down,(1-(1-(x/100)**6)**120)*100)))

ax22 = axs[2].twinx()

axs[2].bar(x_bar, y_bar, color ='blue', width = 9.5, align='edge', log=10)

for v in range(len(final_sum)):
    axs[2].text(x_bar[v]+3.5, np.log10(y_bar[v])*9, str(final_sum[v]), color='orange', fontweight='bold', rotation = 90, size = 20)

ax22.axvline(x=50, color = 'blue', linestyle = ':')
ax22.axvline(x=80, color = 'blue', linestyle = ':')
ax22.axhline(y=85, color = 'blue', linestyle = ':')
ax22.axhline(y=99, color = 'blue', linestyle = ':')


ax22.plot(x, y_outputs)
ax22.text(60,50, 'Tolerance', color='blue', fontweight='bold')
ax22.text(35,35, 'FalsePositive', color='blue', fontweight='bold')
ax22.text(82,101, 'FalseNegative', color='blue', fontweight='bold')
ax22.set_title('(50%,80%,85%,99%)')



plt.show()

# Calculate Large Pairs

In [ ]:
articles_large = pd.read_csv("preproc_art_large.csv")

In [ ]:
encoded_c_p = dict()
encoded_p_c = dict()

In [ ]:
shingles=list()

In [ ]:
list_docs = list(articles_large.article.apply(str))

In [ ]:
for doc in tqdm(list_docs):
    shingles.extend(crete_grams(doc, n=5))

In [ ]:
for gram in tqdm(shingles):
    encode_property('sh',gram)

In [ ]:
articles_large['sh'] = articles_large.article.progress_apply(lambda x: convert_text_in_shingles(str(x), 5))

In [ ]:
large_sign_matrix = create_sign_matr(744, articles_large)

In [ ]:
buckets = get_buckets_LSH(large_sign_matrix, rows = 8, band=93)
pairs = get_candidate_pairs(buckets)
pairs = pd.DataFrame(pairs)

In [ ]:
pairs = pairs.rename(columns={0: 'doc_id1', 1: 'doc_id2'})

In [ ]:
pairs.to_csv('result.csv', index = False)